# 准备config配置文件

同济子豪兄 2023-2-13 6-11

## 进入MMSegmentation主目录

In [1]:
import os
os.chdir('../mmsegmentation')

In [3]:
os.getcwd()

'/home/featurize/work/两天搞定语义分割毕业设计/mmsegmentation'

## 导入工具包

In [4]:
import numpy as np
from PIL import Image

import os.path as osp
from tqdm import tqdm

import mmcv
import mmengine
import matplotlib.pyplot as plt
%matplotlib inline

## 定义数据集类（各类别名称及配色）

In [5]:
!rm -rf mmseg/datasets/DubaiDataset.py # 删除原有文件
!wget https://zihao-openmmlab.obs.cn-east-3.myhuaweicloud.com/20230130-mmseg/Dubai/DubaiDataset.py -P mmseg/datasets


--2023-06-12 10:07:10--  https://zihao-openmmlab.obs.cn-east-3.myhuaweicloud.com/20230130-mmseg/Dubai/DubaiDataset.py
正在连接 172.16.0.13:5848... 已连接。
已发出 Proxy 请求，正在等待回应... 200 OK
长度： 867 [binary/octet-stream]
正在保存至: “mmseg/datasets/DubaiDataset.py”

DubaiDataset.py     100%[===================>]     867  --.-KB/s    用时 0s      

2023-06-12 10:07:10 (21.0 MB/s) - 已保存 “mmseg/datasets/DubaiDataset.py” [867/867])



## 注册数据集类

In [6]:
!rm -rf mmseg/datasets/__init__.py # 删除原有文件
!wget https://zihao-openmmlab.obs.cn-east-3.myhuaweicloud.com/20230130-mmseg/Dubai/__init__.py -P mmseg/datasets


--2023-06-12 10:07:10--  https://zihao-openmmlab.obs.cn-east-3.myhuaweicloud.com/20230130-mmseg/Dubai/__init__.py
正在连接 172.16.0.13:5848... 已连接。
已发出 Proxy 请求，正在等待回应... 200 OK
长度： 2620 (2.6K) [binary/octet-stream]
正在保存至: “mmseg/datasets/__init__.py”

__init__.py         100%[===================>]   2.56K  --.-KB/s    用时 0s      

2023-06-12 10:07:11 (67.5 MB/s) - 已保存 “mmseg/datasets/__init__.py” [2620/2620])



## 定义训练及测试pipeline

In [7]:
!rm -rf configs/_base_/datasets/DubaiDataset_pipeline.py
!wget https://zihao-openmmlab.obs.cn-east-3.myhuaweicloud.com/20230130-mmseg/Dubai/DubaiDataset_pipeline.py -P configs/_base_/datasets


--2023-06-12 10:07:11--  https://zihao-openmmlab.obs.cn-east-3.myhuaweicloud.com/20230130-mmseg/Dubai/DubaiDataset_pipeline.py
正在连接 172.16.0.13:5848... 已连接。
已发出 Proxy 请求，正在等待回应... 200 OK
长度： 2268 (2.2K) [binary/octet-stream]
正在保存至: “configs/_base_/datasets/DubaiDataset_pipeline.py”

DubaiDataset_pipeli 100%[===================>]   2.21K  --.-KB/s    用时 0s      

2023-06-12 10:07:11 (61.6 MB/s) - 已保存 “configs/_base_/datasets/DubaiDataset_pipeline.py” [2268/2268])



## 下载模型config配置文件

In [8]:
!rm -rf configs/pspnet/pspnet_r50-d8_4xb2-40k_DubaiDataset.py # 删除原有文件
!wget https://zihao-openmmlab.obs.cn-east-3.myhuaweicloud.com/20230130-mmseg/Dubai/pspnet_r50-d8_4xb2-40k_DubaiDataset.py -P configs/pspnet 


--2023-06-12 10:07:12--  https://zihao-openmmlab.obs.cn-east-3.myhuaweicloud.com/20230130-mmseg/Dubai/pspnet_r50-d8_4xb2-40k_DubaiDataset.py
正在连接 172.16.0.13:5848... 已连接。
已发出 Proxy 请求，正在等待回应... 200 OK
长度： 344 [binary/octet-stream]
正在保存至: “configs/pspnet/pspnet_r50-d8_4xb2-40k_DubaiDataset.py”

pspnet_r50-d8_4xb2- 100%[===================>]     344  --.-KB/s    用时 0s      

2023-06-12 10:07:12 (8.95 MB/s) - 已保存 “configs/pspnet/pspnet_r50-d8_4xb2-40k_DubaiDataset.py” [344/344])



## 载入config配置文件

In [9]:
from mmengine import Config
cfg = Config.fromfile('./configs/pspnet/pspnet_r50-d8_4xb2-40k_DubaiDataset.py')

## 修改config配置文件

In [10]:
cfg.norm_cfg = dict(type='BN', requires_grad=True) # 只使用GPU时，BN取代SyncBN
cfg.crop_size = (256, 256)
cfg.model.data_preprocessor.size = cfg.crop_size
cfg.model.backbone.norm_cfg = cfg.norm_cfg
cfg.model.decode_head.norm_cfg = cfg.norm_cfg
cfg.model.auxiliary_head.norm_cfg = cfg.norm_cfg
# modify num classes of the model in decode/auxiliary head

# 模型 decode/auxiliary 输出头，指定为类别个数
cfg.model.decode_head.num_classes = 6
cfg.model.auxiliary_head.num_classes = 6

cfg.train_dataloader.batch_size = 8

cfg.test_dataloader = cfg.val_dataloader

# 结果保存目录
cfg.work_dir = './work_dirs/DubaiDataset'

# 训练迭代次数
cfg.train_cfg.max_iters = 3000
# 评估模型间隔
cfg.train_cfg.val_interval = 400
# 日志记录间隔
cfg.default_hooks.logger.interval = 100
# 模型权重保存间隔
cfg.default_hooks.checkpoint.interval = 1500

# 随机数种子
cfg['randomness'] = dict(seed=0)

## 查看完整config配置文件

In [11]:
# print(cfg.pretty_text)

## 保存config配置文件

In [12]:
cfg.dump('pspnet-DubaiDataset_20230612.py')